# Day 16: Handling Imbalanced Data & Model Evaluation

This notebook focuses on solving one of the most common real-world machine learning challenges — **class imbalance** — and understanding how to evaluate models effectively using appropriate performance metrics.

---

## 📌 Learning Objectives
By the end of this session, I was able to:
- Understand the impact of class imbalance on ML models.
- Apply over-sampling and under-sampling techniques.
- Learn how ADASYN improves upon SMOTE.
- Use Tomek Links and CNN for data cleaning.
- Choose the right evaluation metrics for classification and regression problems.

---

## 🚩 Why Class Imbalance Matters
In domains like fraud detection, medical diagnosis, and churn prediction, minority classes are often more important but underrepresented.

➡️ Models trained on such data may show high accuracy but perform poorly on the minority class.  
➡️ Hence, handling imbalance is crucial for building **robust and fair models**.

---

## 🔹 Over-Sampling Techniques

### 🔸 SMOTE — Synthetic Minority Over-sampling Technique
- Creates synthetic samples between minority data points and their neighbors.
- Treats all minority samples equally.

### 🔸 ADASYN — Adaptive Synthetic Sampling
ADASYN extends SMOTE by generating more synthetic samples for **hard-to-learn** minority points near the decision boundary.

#### ⚙️ How ADASYN Works
1. Apply **KNN (k = 5)** on the dataset.
2. For each minority sample, compute a **difficulty score**:

\[
r_i = \frac{\#\text{majority neighbors}}{k}
\]

3. Normalize all \( r_i \) so that \( \sum r_i = 1 \).
4. Generate synthetic samples proportional to normalized \( r_i \).

**Example:**
- p1: 0/3 → r = 0
- p2: 2/3 → r ≈ 0.67
- p3: 3/3 → r = 1

- Normalized: [0, 0.67, 1] → [0, 0.4, 0.6]


➡️ More samples are generated for points closer to the class boundary.

####  ADASYN Procedure
- Use the full dataset.
- Find k-nearest neighbors.
- Compute difficulty scores.
- Normalize scores.
- Generate adaptive synthetic samples.

 Result: Better learning around complex decision boundaries.

---

## 🔹 Under-Sampling Techniques

Under-sampling reduces the number of majority class samples to balance the dataset.

### 🔸 Random Under-Sampling
- Randomly removes majority samples.
- ❌ Risk: loss of important information → weaker decision boundary.

➡️ Prefer **cleaning-based under-sampling** methods.

---

## 🔹 Tomek Links — Boundary Cleaning

A **Tomek link** is a pair of samples (a, b) such that:
- a is the nearest neighbor of b,
- b is the nearest neighbor of a,
- a and b belong to different classes.

➡️ Remove the **majority class** sample from each pair.

**Benefits:**
- Removes overlapping/noisy samples.
- Improves class separability.
- Sharpens the decision boundary.

---

## 🔹 Condensed Nearest Neighbor (CNN)

- Retains samples close to the decision boundary.
- Removes majority samples far from it.
- Produces a compact and informative dataset.

###  Trade-off of Under-Sampling
- ✔️ Faster training, simpler models.
- ❌ Possible **information loss**.

**Summary:**
- Tomek Links → clean noisy boundary points.  
- CNN → retain only critical boundary samples.

---

##  Model Evaluation Metrics

Choosing the right metric is essential for trustworthy models.

### Classification Metrics
- Accuracy  
- Precision  
- Recall  
- F1-score  
- ROC Curve  
- AUC  

➡️ For imbalanced data, **Precision, Recall, F1-score, and AUC-ROC** are more informative than accuracy.

---

### ✅ Regression Metrics
- R² Score  
- Adjusted R² (via `statsmodels`)  
- MAE — Mean Absolute Error  
- MSE — Mean Squared Error  
- RMSE — Root Mean Squared Error  
- MAPE — Mean Absolute Percentage Error  

➡️ **R²** is widely used for evaluation.  
➡️ **MSE/RMSE** are commonly used as loss functions during training.

---

##  Errors as Measures of Dispersion

Errors represent how far predictions deviate from actual values, similar to statistical dispersion.

Mean:

\[
\mu = \frac{1}{n} \sum_{i=1}^{n} x_i
\]

---

##  Key Takeaways
- Class imbalance can heavily bias ML models.
- **ADASYN** adaptively strengthens minority learning near boundaries.
- **Tomek Links & CNN** clean and condense majority samples.
- Proper metrics ensure reliable model evaluation.

>  *Handling imbalance and choosing the right metrics are key skills for building production-ready ML systems.*

---


In [43]:
from sklearn.datasets import load_breast_cancer

In [44]:
import pandas as pd
import numpy as np

In [45]:
dataset = load_breast_cancer()
dataset.keys()

dict_keys(['data', 'target', 'frame', 'target_names', 'DESCR', 'feature_names', 'filename', 'data_module'])

In [46]:
X = dataset.data
y = dataset.target
X.shape, y.shape

((569, 30), (569,))

In [47]:
data = pd.DataFrame(X, columns = dataset.feature_names)
data['target'] = y
data.head()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,target
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,0
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,0
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,0
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,0
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,0


In [48]:
data.shape

(569, 31)

In [49]:
data['target'].value_counts(normalize = True)

target
1    0.627417
0    0.372583
Name: proportion, dtype: float64

In [50]:
X = data.drop('target', axis = 1)
y = data['target']

In [51]:
from sklearn.model_selection import train_test_split

In [52]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42, stratify = y)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((455, 30), (114, 30), (455,), (114,))

### Apply Imbalanced data handling procedures on TRAIN DATA

In [53]:
y_train.value_counts()

target
1    285
0    170
Name: count, dtype: int64

In [55]:
from sklearn.preprocessing import MinMaxScaler

In [57]:
minmax = MinMaxScaler().set_output(transform = 'pandas')
minmax

MinMaxScaler()

In [59]:
X_train_transformed = minmax.fit_transform(X_train)
X_train_transformed.head()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension
546,0.158029,0.224552,0.148711,0.077011,0.387273,0.093737,0.023711,0.027311,0.416667,0.253791,...,0.131746,0.259861,0.115769,0.061507,0.389391,0.059309,0.035016,0.081821,0.219988,0.124295
432,0.624686,0.332432,0.622003,0.469752,0.617958,0.397276,0.499766,0.625746,0.335354,0.222620,...,0.559524,0.347814,0.534351,0.398485,0.647490,0.258957,0.423962,0.746735,0.289178,0.168634
174,0.174121,0.183970,0.163776,0.087497,0.309162,0.072511,0.000000,0.000000,0.438384,0.206192,...,0.143254,0.191365,0.127397,0.068714,0.247436,0.039410,0.000000,0.000000,0.225705,0.043290
221,0.311373,0.141698,0.309585,0.177372,0.518190,0.306178,0.184161,0.221223,0.455556,0.275274,...,0.279762,0.136194,0.283358,0.154429,0.451199,0.235284,0.205831,0.312371,0.295683,0.175325
289,0.207724,0.310450,0.196116,0.107196,0.299550,0.094166,0.056209,0.108002,0.481313,0.201980,...,0.175794,0.376333,0.161440,0.084422,0.275963,0.067711,0.060136,0.213162,0.335502,0.097731


### ADASYN

In [54]:
from imblearn.over_sampling import ADASYN

In [60]:
adasyn = ADASYN()
adasyn

ADASYN()

In [61]:
X_train_res, y_train_res = adasyn.fit_resample(X_train_transformed, y_train)
X_train_res.shape, y_train_res.shape

((574, 30), (574,))

In [62]:
y_train_res.value_counts()

target
0    289
1    285
Name: count, dtype: int64

why 289? some percentage like 0.12 like that

### Undersampling

In [63]:
from imblearn.under_sampling import RandomUnderSampler

In [64]:
rus = RandomUnderSampler()
rus

RandomUnderSampler()

In [65]:
X_train_res, y_train_res = rus.fit_resample(X_train, y_train)
X_train_res.shape, y_train_res.shape

((340, 30), (340,))

In [66]:
y_train_res.value_counts()

target
0    170
1    170
Name: count, dtype: int64

### Tomek Links

In [67]:
from imblearn.under_sampling import TomekLinks

In [68]:
tl = TomekLinks()
tl

TomekLinks()

In [69]:
X_train_res, y_train_res = tl.fit_resample(X_train_transformed, y_train)
X_train_res.shape, y_train_res.shape

((450, 30), (450,))

In [70]:
y_train_res.value_counts()

target
1    280
0    170
Name: count, dtype: int64

In [71]:
tl = TomekLinks(sampling_strategy = 'all')
tl

TomekLinks(sampling_strategy='all')

In [72]:
X_train_res, y_train_res = tl.fit_resample(X_train_transformed, y_train)
X_train_res.shape, y_train_res.shape

((445, 30), (445,))

In [73]:
y_train_res.value_counts()

target
1    280
0    165
Name: count, dtype: int64

### Condensed Nearest Neighbors

In [74]:
from imblearn.under_sampling import CondensedNearestNeighbour

In [75]:
cnn = CondensedNearestNeighbour()
cnn

CondensedNearestNeighbour()

In [76]:
X_train_res, y_train_res = cnn.fit_resample(X_train_transformed, y_train)
X_train_res.shape, y_train_res.shape

((206, 30), (206,))

In [77]:
y_train_res.value_counts()

target
0    170
1     36
Name: count, dtype: int64

#### “Applied ADASYN and under-sampling techniques on the Breast Cancer dataset to handle class imbalance and studied their impact on class distribution.”